# Задание 10.

Вам предоставлен набор данных по качеству воздуха в китайских городах в виде csv файлов.
Основная метрика, которая показывает качество воздуха - PM25(<a href='https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4740125/'>подробнее о PM25</a>), указана в колонках, которые начинаются на PM_.
Остальные столбцы содержат либо данные о времени, в которое были произведены измерения, либо данные о погоде в этот момент. DEWP - точка росы, HUMI - влажность, PRES - давление, TEMP - температура, cbwd - направление ветра, Iws - его скорость, precipation - осадки, Iprec - накопленные осадки.

Для выполнения задания необходимо ответить на вопросы по данным, заполнив ячейки ноутбука. Сам ноутбук необходимо выложить в виде отдельного репозитория на github. Ссылку для клонирования репозитория необходимо приложить в решение задания.

## 1. Загрузите в виде DataFrame набор данных по качеству воздуха Пекина
Ссылка на набор данных: https://disk.yandex.ru/d/1UKJoV1ULW1NLQ

In [1]:
import requests
from urllib.parse import urlencode
import pandas as pd

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/1UKJoV1ULW1NLQ'

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

download_response = requests.get(download_url)
with open('downloaded_file.csv', 'wb') as f:
    f.write(download_response.content)
file = pd.read_csv("downloaded_file.csv")

## 2. Сколько всего строк находится в наборе данных?

In [2]:
file.shape[0]

52584

## 3. Сколько измерений уровня загрязнения воздуха находится в наборе данных?

In [3]:
#file.loc[:, ["PM_Dongsi", "PM_Dongsihuan", "PM_Nongzhanguan", "PM_US Post"]].dropna(how="all").shape[0]
file.loc[:, ["PM_Dongsi", "PM_Dongsihuan", "PM_Nongzhanguan", "PM_US Post"]].count().sum()
#первый вариант подходит, если за измерение считать строчку с минимум одним не пустым значением в столбцах PM.
#второй вариант подходит, если за измерение считать не пустую ячейку в столбцах PM.

120878

## 4. Введите столбец среднего качества воздуха PM_mean с усреднением показаний со всех станций слежения.

In [4]:
file.insert(10, "PM_mean", file.loc[:, ["PM_Dongsi", "PM_Dongsihuan", "PM_Nongzhanguan", "PM_US Post"]].mean(1))

## 5. Какое направление ветра преобладает в Пекине?

In [5]:
file.loc[:, ["cbwd"]].mode()

,cbwd
0,SE


## 6. В какой день качество воздуха является наихудшим(самая большая концентрация частиц)? Вывести дату в типе данных datetime стандартной библиотеки Python.

In [6]:
from datetime import datetime
max_row_id = file["PM_mean"].idxmax()
year, mounth, day, hour = file.iloc[max_row_id, [1, 2, 3, 4]].tolist()
datetime(year, mounth, day, hour)

datetime.datetime(2012, 1, 23, 0, 0)

## 7. Какой из сезонов в Пекине в среднем наименее неблагоприятный для легких за последние 2 года наблюдений?

In [7]:
file[file.year > 2013].groupby("season").mean()["PM_mean"].idxmin()

2

## 8. Коррелирует ли влажность с качеством воздуха?
Можно воспользоваться корреляцией Пирсона из numpy, а также встроенными функциями Pandas.

In [8]:
file.corr().loc["HUMI", "PM_mean"]

0.42411474824297307

## 9. Какой из сезонов в Пекине самый дождливый(в среднем по накоплению осадков)?

In [9]:
file.groupby("season").mean()["Iprec"].idxmax()

3

## 10. Становится ли качество воздуха во время дождя лучше, чем оно было сразу перед дождем?

In [10]:
before_rain = pd.DataFrame(columns=file.columns)
after_rain = pd.DataFrame(columns=file.columns)
for index, row in file.iterrows():
    if row["precipitation"] == 0:
        before = row
    if (row["precipitation"] != 0) and (before["precipitation"] == 0):
        after_rain = after_rain.append(row,  ignore_index=True)
        before_rain = before_rain.append(before, ignore_index=True)
        before = row
    elif row["precipitation"] != 0:
        after_rain = after_rain.append(row, ignore_index=True)
if before_rain["PM_mean"].mean()>after_rain["PM_mean"].mean():
    print("Да")
else:
    print("Нет")

Да
